In [1]:
#Team7 - Project2 Submission - Machine Learning

        #Rachakonda, Sravan Kumar (1001824673)
        #Mallina, Sai Harshita (1001816947)
        #Manchireddy, Apoorva (1001820533)

#Make Sure that all files need to be placed in one folder.
#Make Sure the Auto-downloading of the dataset file is downloading to current folder.
#Otherwise, Copy the dataset folder to current running folder.

#If there any changes to file name of DataSet, please use the below link to download the dataset file:
#    https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [2]:
#Downloading the DataSet File.Takes time 
import urllib.request
import tarfile

url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' #Data Set URL
download_filename = 'aclImdb_v1.tar.gz' #File Name to be downloaded
urllib.request.urlretrieve(url, download_filename) #Downloading the requested File
filename = 'aclImdb_v1.tar.gz'
if filename.endswith("tar.gz"):  #Unzipping the tar file
    tar = tarfile.open(filename, "r:gz")
    tar.extractall()
    tar.close()

In [3]:
#Importing packages and libraries
import re
import glob
import pandas as pd
from pandas import DataFrame

#Importing NLTK library for text pre-processing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\srava\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srava\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#getNamesOfFilesFromPath function takes path and returns the all the names of requested pattern files in that path.
def getNamesOfFilesFromPath(path):
    filenames = glob.glob(path) #go to that path and returns the list of file names
    return filenames

In [5]:
#getReviews function takes the list of names of files and returns the list of content of the files
def getReviews(myList = [], *args):
    listReview=[]
    for file in myList: #iterating through each file name
        fileHandle = open(file,"rb") #opening the file
        contents=fileHandle.read() #reading the content of the file
        listReview.append(str(contents)[2:]) #storing the file contents to a list
    return listReview # returning the whole list

In [6]:
#getRatings function takes the list of names of files and returns the list of (ratings, reviewNumbers)
def getRatings(myList = [], *args):
    ratings=[]
    for name in myList: #iterating through each file name
        ratings.append(re.split("[_.]",name)) #spliting the file name and getting rating, reviewNumber and adding to list
    return ratings #returing the list

In [7]:
#createDataFrame function creates a DataFrame of requested parameters: reviews, dfRatings, category, typeOfData
def createDataFrame(reviews, dfRatings, category, typeOfData):
    if typeOfData == 'train': #Enters this if typeOfData is train
        dfRatings[0] = [rating[18:] for rating in dfRatings[0]] #grepping ratings
    elif typeOfData == 'test': #Enters this if typeOfData is test
        dfRatings[0] = [rating[17:] for rating in dfRatings[0]] #grepping ratings
    df=DataFrame(reviews,columns=['Reviews']) #creating a Reviews column in the dataframe
    df['Category'] = category #creating a Category column in the dataframe
    df['reviewNumbers']=dfRatings[0] #creating a reviewNumbers column in the dataframe
    df['ratings']=dfRatings[1] #creating a ratings column in the dataframe
    df=df.reindex(columns=['reviewNumbers','Category','Reviews','ratings'])
    return df

In [8]:
#Cleaning - Removing punctuations
def removePunctuations(df):
    remPunctNoSpace = re.compile("[.;:*~!\'?,\"()\\\[\]]") #matches with nospace 
    remPunctWithSpace = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)") #matches with space
    df = [remPunctNoSpace.sub("", i.lower()) for i in df] 
    df = [remPunctWithSpace.sub(" ", i) for i in df]
    return df

In [9]:
#PreProcessing - Removing stopwords from data
stopWords = stopwords.words('english') #Gets all the stopwords
def removeStopwords(df):
    tempList = []
    for i in df:
        tempList.append(' '.join([j for j in i.split() 
                                       if j not in stopWords])) #removing the stopwords if they exists in the given data
    return tempList

In [10]:
#PreProcessing - Lemmatization from data
def Lemmatization(dataList):
    newDataList=[' '.join([WordNetLemmatizer().lemmatize(i) for i in j.split()]) for j in dataList]
    return newDataList

In [11]:
#Reading the positive reviews file names from train folder
trainPos = getNamesOfFilesFromPath("aclImdb/train/pos/*.txt")

In [12]:
#Merging all positive review txt files from train folder and storing them in a dataframe

trainPosReview = getReviews(trainPos) #getting the content of positive reviews from files placed in train folder
trainPosRatings = getRatings(trainPos) #getting the list of ratings, reviewNumber

dfTrainPosRatings=DataFrame(trainPosRatings) #converting the ratings list to dataframe
dfTrainPos = createDataFrame(trainPosReview, dfTrainPosRatings, 'positive', 'train') #creates a dataframe for positive reviews train data

dfTrainPos

,reviewNumbers,Category,Reviews,ratings
0,0,positive,Bromwell High is a cartoon comedy. It ran at t...,9
1,10000,positive,Homelessness (or Houselessness as George Carli...,8
2,10001,positive,Brilliant over-acting by Lesley Ann Warren. Be...,10
3,10002,positive,This is easily the most underrated film inn th...,7
4,10003,positive,This is not the typical Mel Brooks film. It wa...,8
...,...,...,...,...
12495,9998,positive,"Seeing as the vote average was pretty low, and...",9
12496,9999,positive,"The plot had some wretched, unbelievable twist...",8
12497,999,positive,I am amazed at how this movie(and most others ...,10
12498,99,positive,A Christmas Together actually came before my t...,8


In [13]:
#Reading the negative reviews from train folder
trainNeg = getNamesOfFilesFromPath("aclImdb/train/neg/*.txt")

In [14]:
#Merging all negative review txt files from train folder and storing them in a dataframe
trainNegReview = getReviews(trainNeg) #getting the content of negative reviews from files placed in train folder
trainNegRatings = getRatings(trainNeg) #getting the list of ratings, reviewNumber

dfTrainNegRatings = DataFrame(trainNegRatings) #converting the ratings list to dataframe
dfTrainNeg = createDataFrame(trainNegReview, dfTrainNegRatings, 'negative', 'train') #creates a dataframe for negative reviews train data
dfTrainNeg

,reviewNumbers,Category,Reviews,ratings
0,0,negative,Story of a man who has unnatural feelings for ...,3
1,10000,negative,Airport '77 starts as a brand new luxury 747 p...,4
2,10001,negative,This film lacked something I couldn't put my f...,4
3,10002,negative,"Sorry everyone,,, I know this is supposed to b...",1
4,10003,negative,When I was little my parents took me along to ...,1
...,...,...,...,...
12495,9998,negative,"Towards the end of the movie, I felt it was to...",4
12496,9999,negative,This is the kind of movie that my enemies cont...,3
12497,999,negative,I saw 'Descent' last night at the Stockholm Fi...,3
12498,99,negative,Some films that you pick up for a pound turn o...,1


In [15]:
#Reading the positive reviews from test folder
testPos = getNamesOfFilesFromPath("aclImdb/test/pos/*.txt")

In [16]:
#Merging all positive review txt files from test folder and storing them in a dataframe

testPosReview = getReviews(testPos) #getting the content of positive reviews from files placed in test folder
testPosRatings = getRatings(testPos) #getting the list of ratings, reviewNumber

dftestPosRatings=DataFrame(testPosRatings) #converting the ratings list to dataframe
dfTestPos = createDataFrame(testPosReview, dftestPosRatings, 'positive', 'test') #creates a dataframe for positive reviews test data

dfTestPos

,reviewNumbers,Category,Reviews,ratings
0,0,positive,I went and saw this movie last night after bei...,10
1,10000,positive,Actor turned director Bill Paxton follows up h...,7
2,10001,positive,As a recreational golfer with some knowledge o...,9
3,10002,positive,"I saw this film in a sneak preview, and it is ...",8
4,10003,positive,Bill Paxton has taken the true story of the 19...,8
...,...,...,...,...
12495,9998,positive,I was extraordinarily impressed by this film. ...,8
12496,9999,positive,"Although I\'m not a golf fan, I attended a sne...",10
12497,999,positive,"From the start of ""The Edge Of Love"", the view...",8
12498,99,positive,"This movie, with all its complexity and subtle...",10


In [17]:
#Reading the negative reviews from test folder
testNeg = getNamesOfFilesFromPath("aclImdb/test/neg/*.txt")

In [18]:
#Merging all negative review txt files from test folder and storing them in a dataframe
testNegReview = getReviews(testNeg) #getting the content of negative reviews from files placed in test folder
testNegRatings = getRatings(testNeg) #getting the list of ratings, reviewNumber

dfTestNegRatings = DataFrame(testNegRatings) #converting the ratings list to dataframe
dfTestNeg = createDataFrame(testNegReview, dfTestNegRatings, 'negative', 'test') #creates a dataframe for negative reviews test data
dfTestNeg

,reviewNumbers,Category,Reviews,ratings
0,0,negative,Once again Mr. Costner has dragged out a movie...,2
1,10000,negative,This is an example of why the majority of acti...,4
2,10001,negative,"First of all I hate those moronic rappers, who...",1
3,10002,negative,Not even the Beatles could write songs everyon...,3
4,10003,negative,Brass pictures (movies is not a fitting word f...,3
...,...,...,...,...
12495,9998,negative,I occasionally let my kids watch this garbage ...,1
12496,9999,negative,When all we have anymore is pretty much realit...,1
12497,999,negative,The basic genre is a thriller intercut with an...,3
12498,99,negative,Four things intrigued me as to this film - fir...,3


In [19]:
#Combining both postive and negative reviews for train and test data
dfTrain = dfTrainPos.append(dfTrainNeg) # adding positive train and negative train data to a single varaible

dfTest = dfTestPos.append(dfTestNeg) # adding positive train and negative test data to a single varaible

In [20]:
#Cleaning - Removing punctuations from train and test data

dfTrain['Reviews after Cleaning'] = removePunctuations(dfTrain['Reviews']) #removes punctuations 
dfTest['Reviews after Cleaning'] = removePunctuations(dfTest['Reviews']) #removes punctuations 

In [21]:
#PreProcessing - Removing stopwords from cleaned train data
AfterStopWordsRemovalTrain = removeStopwords(dfTrain['Reviews after Cleaning'])

AfterStopWordsRemovalTrain

['bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt',
 'homelessness houselessness george carlin stated issue years never plan help street considered human everything going school work vote matter people think homeless lost cause worrying things racism war iraq pressuring kids succeed technology elections inflation worrying theyll next end streets given bet live streets month without luxuries home entertainment sets bathroom pictures wall computer everything treasure see like homeless goddard bolts lesson mel brooks directs stars bo

In [22]:
#PreProcessing - Lemmatization for train data

LemmatizedTrain= Lemmatization(AfterStopWordsRemovalTrain)

LemmatizedTrain

['bromwell high cartoon comedy ran time program school life teacher 35 year teaching profession lead believe bromwell high satire much closer reality teacher scramble survive financially insightful student see right pathetic teacher pomp pettiness whole situation remind school knew student saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teacher student welcome bromwell high expect many adult age think bromwell high far fetched pity isnt',
 'homelessness houselessness george carlin stated issue year never plan help street considered human everything going school work vote matter people think homeless lost cause worrying thing racism war iraq pressuring kid succeed technology election inflation worrying theyll next end street given bet live street month without luxury home entertainment set bathroom picture wall computer everything treasure see like homeless goddard bolt lesson mel brook directs star bolt play rich man everyth

In [23]:
#PreProcessing - Removing stopwords from cleaned test data

AfterStopWordsRemovalTest = removeStopwords(dfTest['Reviews after Cleaning'])

AfterStopWordsRemovalTest

['went saw movie last night coaxed friends mine ill admit reluctant see knew ashton kutcher able comedy wrong kutcher played character jake fischer well kevin costner played ben randall professionalism sign good movie toy emotions one exactly entire theater sold overcome laughter first half movie moved tears second half exiting theater saw many women tears many full grown men well trying desperately let anyone see crying movie great suggest go see judge',
 'actor turned director bill paxton follows promising debut gothic horror frailty family friendly sports drama 1913 us open young american caddy rises humble background play bristish idol dubbed greatest game ever played im fan golf scrappy underdog sports flicks dime dozen recently done grand effect miracle cinderella man film enthralling film starts creative opening credits imagine disneyfied version animated opening credits hbos carnivale rome lumbers along slowly first numbers hour action moves us open things pick well paxton nice

In [24]:
#PreProcessing - Lemmatization for test data

LemmatizedTest = Lemmatization(AfterStopWordsRemovalTest)

LemmatizedTest

['went saw movie last night coaxed friend mine ill admit reluctant see knew ashton kutcher able comedy wrong kutcher played character jake fischer well kevin costner played ben randall professionalism sign good movie toy emotion one exactly entire theater sold overcome laughter first half movie moved tear second half exiting theater saw many woman tear many full grown men well trying desperately let anyone see cry movie great suggest go see judge',
 'actor turned director bill paxton follows promising debut gothic horror frailty family friendly sport drama 1913 u open young american caddy rise humble background play bristish idol dubbed greatest game ever played im fan golf scrappy underdog sport flick dime dozen recently done grand effect miracle cinderella man film enthralling film start creative opening credit imagine disneyfied version animated opening credit hbos carnivale rome lumber along slowly first number hour action move u open thing pick well paxton nice job show knack effe

In [25]:
dfTrain['Reviews after Cleaning'] = LemmatizedTrain
dfTrain.to_csv('trainAfterCleaningPreprocessing.csv',index=False)

dfTest['Reviews after Cleaning'] = LemmatizedTest
dfTest.to_csv('testAfterCleaningPreprocessing.csv',index=False)

In [26]:
print('Train Dataset after Preprocessing')
dfTrain

Train Dataset after Preprocessing


,reviewNumbers,Category,Reviews,ratings,Reviews after Cleaning
0,0,positive,Bromwell High is a cartoon comedy. It ran at t...,9,bromwell high cartoon comedy ran time program ...
1,10000,positive,Homelessness (or Houselessness as George Carli...,8,homelessness houselessness george carlin state...
2,10001,positive,Brilliant over-acting by Lesley Ann Warren. Be...,10,brilliant acting lesley ann warren best dramat...
3,10002,positive,This is easily the most underrated film inn th...,7,easily underrated film inn brook cannon sure f...
4,10003,positive,This is not the typical Mel Brooks film. It wa...,8,typical mel brook film much le slapstick movie...
...,...,...,...,...,...
12495,9998,negative,"Towards the end of the movie, I felt it was to...",4,towards end movie felt technical felt like cla...
12496,9999,negative,This is the kind of movie that my enemies cont...,3,kind movie enemy content watch time bloody tru...
12497,999,negative,I saw 'Descent' last night at the Stockholm Fi...,3,saw descent last night stockholm film festival...
12498,99,negative,Some films that you pick up for a pound turn o...,1,film pick pound turn rather good 23rd century ...


In [27]:
print('Test Dataset after Preprocessing')
dfTest

Test Dataset after Preprocessing


,reviewNumbers,Category,Reviews,ratings,Reviews after Cleaning
0,0,positive,I went and saw this movie last night after bei...,10,went saw movie last night coaxed friend mine i...
1,10000,positive,Actor turned director Bill Paxton follows up h...,7,actor turned director bill paxton follows prom...
2,10001,positive,As a recreational golfer with some knowledge o...,9,recreational golfer knowledge sport history pl...
3,10002,positive,"I saw this film in a sneak preview, and it is ...",8,saw film sneak preview delightful cinematograp...
4,10003,positive,Bill Paxton has taken the true story of the 19...,8,bill paxton taken true story 1913 u golf open ...
...,...,...,...,...,...
12495,9998,negative,I occasionally let my kids watch this garbage ...,1,occasionally let kid watch garbage understand ...
12496,9999,negative,When all we have anymore is pretty much realit...,1,anymore pretty much reality tv show people mak...
12497,999,negative,The basic genre is a thriller intercut with an...,3,basic genre thriller intercut uncomfortable me...
12498,99,negative,Four things intrigued me as to this film - fir...,3,four thing intrigued film firstly star carly p...
